In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup

Importing all the required libraries for reading the HTML + Cleaning the HTML

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

Checking if the page has been read correctly

In [3]:
title = soup.title
print(title)

<title>List of postal codes of Canada: M - Wikipedia</title>


In [4]:
table = soup.find('table')

In [5]:
rows = table.find_all('td')

In [6]:
str_cells = str(rows)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
print(cleantext)

[M1A, Not assigned, Not assigned
, M2A, Not assigned, Not assigned
, M3A, North York, Parkwoods
, M4A, North York, Victoria Village
, M5A, Downtown Toronto, Harbourfront
, M5A, Downtown Toronto, Regent Park
, M6A, North York, Lawrence Heights
, M6A, North York, Lawrence Manor
, M7A, Queen's Park, Not assigned
, M8A, Not assigned, Not assigned
, M9A, Etobicoke, Islington Avenue
, M1B, Scarborough, Rouge
, M1B, Scarborough, Malvern
, M2B, Not assigned, Not assigned
, M3B, North York, Don Mills North
, M4B, East York, Woodbine Gardens
, M4B, East York, Parkview Hill
, M5B, Downtown Toronto, Ryerson
, M5B, Downtown Toronto, Garden District
, M6B, North York, Glencairn
, M7B, Not assigned, Not assigned
, M8B, Not assigned, Not assigned
, M9B, Etobicoke, Cloverdale
, M9B, Etobicoke, Islington
, M9B, Etobicoke, Martin Grove
, M9B, Etobicoke, Princess Gardens
, M9B, Etobicoke, West Deane Park
, M1C, Scarborough, Highland Creek
, M1C, Scarborough, Rouge Hill
, M1C, Scarborough, Port Union
, M2C

Cleaning the data table and converting it into a DataFrame

In [7]:
list = cleantext.split("\n")
df = pd.DataFrame(list)
df1 = df[0].str.split(',', expand=True)
df1.head(10)

,0,1,2,3
0,[M1A,Not assigned,Not assigned,None
1,,M2A,Not assigned,Not assigned
2,,M3A,North York,Parkwoods
3,,M4A,North York,Victoria Village
4,,M5A,Downtown Toronto,Harbourfront
5,,M5A,Downtown Toronto,Regent Park
6,,M6A,North York,Lawrence Heights
7,,M6A,North York,Lawrence Manor
8,,M7A,Queen's Park,Not assigned
9,,M8A,Not assigned,Not assigned


In [8]:
#Naming the columns as required
df1 = df1.drop(columns=0)
df1.iloc[0,0] = 'M1A'
df1.columns=["PostalCode","Borough","Neighbourhood"]
df1.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,None
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
#Deleting Rows where the Borough is not assigned
df2=df1
df2['Borough'] = df2['Borough'].str.strip()
df3 = df2
df3.drop(df3[df3.Borough == 'Not assigned'].index, inplace=True)

In [11]:
df3.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [12]:
#for Neighrbourhoods which are not assigned, assigning the value of the Borough
df3['Neighbourhood'] = df3['Neighbourhood'].str.strip()
df3.loc[df3['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df3['Borough']

(212, 3)

In [13]:
df3.drop(df3.index[[211]], inplace=True)

# Final Shape of the DataFrame after cleaning is (211, 3)

In [14]:
#Part 1 - Final Shape of the Required Data Frame
df3.shape

(211, 3)

In [15]:
df4 = df3.reset_index(drop=True)

In [16]:
latlon = pd.read_csv('http://cocl.us/Geospatial_data')

In [17]:
latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
df4.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [18]:
type(latlon.iloc[0,0])

str

In [42]:
type(df4.iloc[0,1])

str

In [38]:
latlon['Postal Code'] = latlon['Postal Code'].str.lstrip()

In [41]:
df4['PostalCode'] = df4['PostalCode'].str.strip()
value = latlon.iloc[0,0]
value

'M1B'

In [53]:
merged_left = pd.merge(left=df4,right=latlon, left_on='PostalCode', right_on='Postal Code', how='left', indicator=True)

In [54]:
merged_left.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,_merge
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,both
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,both
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636,both
3,M5A,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636,both
4,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763,both


In [45]:
merged_left

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,_merge
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,both
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,both
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636,both
3,M5A,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636,both
4,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763,both
5,M6A,North York,Lawrence Manor,M6A,43.718518,-79.464763,both
6,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494,both
7,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242,both
8,M1B,Scarborough,Rouge,M1B,43.806686,-79.194353,both
9,M1B,Scarborough,Malvern,M1B,43.806686,-79.194353,both


In [57]:
type(merged_left)

pandas.core.frame.DataFrame

In [56]:
merged_left.drop(["Postal Code", "_merge"], axis=1, inplace=True)

In [58]:
merged_left.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
